In [2]:
import os as os
import pandas as pd

os.chdir('../../Google Drive/elections_niger')

In [3]:
missing_bureaux = pd.read_csv('data/bureaux_absents.csv')

In [4]:
len(missing_bureaux)

800

In [5]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import datetime as datetime
import time

In [6]:
to_check = missing_bureaux[missing_bureaux.ID_COMMUNE.isin([79001 , 59002 , 79004 , 59001 , 79002 ,79003 , 79005 , 70506])]

In [8]:
len(to_check)

500

In [44]:
def extract_list(url) :
    lu  = 'non'
    while lu == 'non' :
        try :
            r = requests.get(url)
            lu = 'oui'
        except :
            print('On réessaye')
    r = r.text
    soup = BeautifulSoup(r , 'html.parser') 
    table_results =  soup.select("table") 
    table_rows = table_results[0].findAll('tr')
    nrows = len(table_rows)
    
    dframe = {}
    
    for row in range(1 , nrows) :
        cont = table_rows[row].findAll('td')
        row_val = []
        ncol = len(cont)
        for column in range(ncol) :
            row_val = row_val + [cont[column].text.strip()]

        dframe[str(row)] = row_val

    out = pd.DataFrame(dframe).T
    out 
    return out

In [12]:
list_communes = pd.read_csv('data/Niger_Communes.csv')

In [46]:
list_communes.head()

,Unnamed: 0,ID_COMMUNE,ID_DEPART,NOM_COMMUNE,N_COMMUNE
0,0,10101,101,ADERBISSINAT,NaN
1,1,10201,102,ARLIT,NaN
2,2,10202,102,DANNET,NaN
3,3,10203,102,GOUGARAM,NaN
4,4,10301,103,BILMA,NaN


In [9]:
n_communes = to_check.ID_COMMUNE.nunique()
communes = to_check.ID_COMMUNE.unique()

In [47]:
for i in range(0,n_communes) :
    id_commune = communes[i]
    nom_commune = list_communes.NOM_COMMUNE[list_communes.ID_COMMUNE == communes[i]].to_string(index=False)
    id_depart = list_communes.ID_DEPART[list_communes.ID_COMMUNE == communes[i]].to_string(index=False)
    dat_com = to_check[to_check.ID_COMMUNE == id_commune]
    n_bureaux = len(dat_com)
    n_bureaux_extracted = 0
    print(nom_commune)
    for j in range(n_bureaux):
        id_bureau = dat_com.reset_index().ID_BUREAU[j]
        url = 'http://ceni-niger.org/ceniProject/electeurs/' + str(id_bureau) + '/bureau'
        print(url)
        electeurs = extract_list(url)
        if len(electeurs) > 0:
            electeurs.columns = ['0','1','2','3','4','5','6']
            n_bureaux_extracted = n_bureaux_extracted + 1
        if n_bureaux_extracted == 1 :
            electeurs['ID_BUREAU'] = str(id_bureau)
            out = electeurs
        if n_bureaux_extracted > 1 :
            electeurs['ID_BUREAU'] = str(id_bureau)
            out = out.append(electeurs, ignore_index=False)
    nom_commune = nom_commune.replace('/','-')
    nom_csv = 'data/listes_electeurs/' + str(id_commune) + nom_commune + '.csv'
    print(str(100 * (i+1) / (n_communes)) + ' % extracted')
    out.to_csv(nom_csv)

ARRONDISSEMENT 1
http://ceni-niger.org/ceniProject/electeurs/59001/bureau
http://ceni-niger.org/ceniProject/electeurs/59001001/bureau
http://ceni-niger.org/ceniProject/electeurs/59001002/bureau
http://ceni-niger.org/ceniProject/electeurs/59001003/bureau
http://ceni-niger.org/ceniProject/electeurs/59001004/bureau
http://ceni-niger.org/ceniProject/electeurs/59001005/bureau
http://ceni-niger.org/ceniProject/electeurs/59001006/bureau
http://ceni-niger.org/ceniProject/electeurs/59001007/bureau
http://ceni-niger.org/ceniProject/electeurs/59001008/bureau
http://ceni-niger.org/ceniProject/electeurs/59001009/bureau
http://ceni-niger.org/ceniProject/electeurs/59001010/bureau
http://ceni-niger.org/ceniProject/electeurs/59001011/bureau
http://ceni-niger.org/ceniProject/electeurs/59001012/bureau
http://ceni-niger.org/ceniProject/electeurs/59001013/bureau
http://ceni-niger.org/ceniProject/electeurs/59001014/bureau
http://ceni-niger.org/ceniProject/electeurs/59001015/bureau
http://ceni-niger.org/ceni

In [48]:
list_communes.NOM_COMMUNE.value_counts()

ARRONDISSEMENT 1     3
ARRONDISSEMENT 2     3
ARRONDISSEMENT 5     2
ARRONDISSEMENT 4     2
ARRONDISSEMENT 3     2
OURAFANE             1
DINGAZI              1
BOUZA                1
OURNO                1
KIRTACHI             1
MIRRIAH              1
BENGOU               1
SABON GUIDA          1
MOKKO                1
TONDIKANDIA          1
TANDA                1
GANGARA (AGUIE)      1
DOGO-DOGO            1
TOUMOUR              1
GOUDOUMARIA          1
EGYPTE               1
GUIDAN AMOUMOUNE     1
DIRKOU               1
KANAN-BAKACHE        1
BITINKODJI           1
Italie               1
TESKER               1
MAKALONDI            1
USA                  1
GADABEDJI            1
                    ..
FOULATARI            1
KIOTA                1
TIBIRI (MARADI)      1
SANAM                1
TILLABERI            1
TONDIKIWINDI         1
SINDER               1
MAIYARA              1
MOA                  1
DAKOUSSA             1
EMIRAT ARABE UNIE    1
GUINEE CONAKRY       1
KOLLERAM   